In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]= "2,3"

In [2]:
import torch
model = torch.hub.load("pytorch/vision", "vit_b_32")

import torchvision
from Models.transformer import VisionTransformer as vit
import Models.Conv as conv

from DataLoader import CIFAR100
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

import utils

Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
train_loader, test_loader = CIFAR100.get_data(1024*2, resize=36, crop=32)
# train_loader, test_loader = CIFAR100.get_data(1024*2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
depths = [18, 50, 101]

for depth in depths:
    device_ids = [0, 1]
    device = "cuda"
    class_num = 100
    if depth >= 50:
        model = conv.resnet(class_num= class_num, depth = depth, pretrained = False, weights= "IMAGENET1K_V2")
    else:
        model = conv.resnet(class_num= class_num, depth = depth, pretrained = False, weights= "IMAGENET1K_V1")
 
    model = model.to(device)
    model = torch.nn.DataParallel(model, device_ids=device_ids)

    CE = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)


    train_acc = []
    test_acc = []
    best_acc = 0.0
    stack = 0
    step_epoch = []

    for epoch in range(400):
        print(f"lr : {lr_scheduler.get_last_lr()}")
        acc = utils.train_vanilla(model, train_loader, optimizer, CE, device, epoch)
        train_acc.append(acc.item())

        acc = utils.test(model, test_loader, device, epoch)
        test_acc.append(acc.item())

        if acc > best_acc+ 0.005:
            best_acc = acc
            stack = 0
        else:
            stack+=1

        if stack > 4 + 0.003:
            stack = 0
            lr_scheduler.step()
            step_epoch.append(epoch)
            print("STEP!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        
    torch.save(model, f"saved_models/resnet/resnet_basic_{depth}.pth")
    del model
    torch.cuda.empty_cache()
    
    with open(f"saved_models/resnet/resnet_basic_{depth}.json", "w") as f:
        json.dump({"student_test_accs" : train_acc,
                  "test_acc" : test_acc}, f)

Using cache found in /root/.cache/torch/hub/pytorch_vision_main


lr : [0.01]


100%|██████████| 25/25 [00:11<00:00,  2.22it/s]


0 	 train acc : 0.08845999836921692


100%|██████████| 5/5 [00:02<00:00,  2.45it/s]


...

398 	 test acc : 0.6262999773025513
lr : [1.0000000000000005e-09]


100%|██████████| 25/25 [00:06<00:00,  4.13it/s]


399 	 train acc : 0.6696599721908569


100%|██████████| 5/5 [00:01<00:00,  2.62it/s]


399 	 test acc : 0.6233999729156494


In [4]:
depths = [11, 13, 16, 19]

for depth in depths:
    device_ids = [0, 1]
    device = "cuda"
    class_num = 100

    model = conv.vgg_bn(class_num, depth = depth, pretrained = True, weights= "IMAGENET1K_V1")
    model = model.to(device)
#     model = torch.nn.DataParallel(model, device_ids=device_ids)

    CE = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)


    train_acc = []
    test_acc = []
    best_acc = 0.0
    stack = 0
    step_epoch = []

    for epoch in range(80):
        print(f"lr : {lr_scheduler.get_last_lr()}")
        acc = utils.train_vanilla(model, train_loader, optimizer, CE, device, epoch)
        train_acc.append(acc.item())

        acc = utils.test(model, test_loader, device, epoch)
        test_acc.append(acc.item())

        if acc > best_acc + 0.005:
            best_acc = acc
            stack = 0
        else:
            stack+=1

        if stack > 4:
            stack = 0
            lr_scheduler.step()
            step_epoch.append(epoch)
            print("STEP!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        
    torch.save(model, f"saved_models/vgg/vgg_basic_{depth}.pth")
    del model
    torch.cuda.empty_cache()
    
    with open(f"saved_models/vgg/vgg_basic_{depth}.json", "w") as f:
        json.dump({"student_test_accs" : train_acc,
                  "test_acc" : test_acc}, f)

Using cache found in /root/.cache/torch/hub/pytorch_vision_main


lr : [0.01]


100%|██████████| 25/25 [00:06<00:00,  4.13it/s]


0 	 train acc : 0.050859998911619186


100%|██████████| 5/5 [00:01<00:00,  2.63it/s]


...

78 	 test acc : 0.7080999612808228
lr : [1.0000000000000005e-09]


100%|██████████| 25/25 [00:09<00:00,  2.63it/s]


79 	 train acc : 0.7099399566650391


100%|██████████| 5/5 [00:01<00:00,  2.85it/s]


79 	 test acc : 0.70660001039505


In [11]:
for img, l in train_loader:
    print(img.shape)
    break

torch.Size([2048, 3, 28, 28])


In [1]:
!nvidia-smi

Mon Sep 12 11:36:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   27C    P8    15W / 300W |      0MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:1D:00.0 Off |                  Off |
| 30%   